# Adversarial FedEM Client

TJ Kim

12.22.21

### Summary:
- Add functionality to client in FedEM System that allows for generation of adversarial examples and training on them
- Copy original dataset to the side. The place of old dataset is replaced by the "fractionally adversarial dataset"
- Random sampling of adversarial data points, remember idx so that it can be replaced in original location
- Input --  fraction of dataset to be adversarial, slider on how much of the dataset is perturbed
- Combining weighted hypotheses into a single adversarial neural network 

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


#### Import Libraries

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

#### Generate Items for ADV Client

In [3]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedEM"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_12_02_first_transfers_xadv_train_n40/'
args_.validation = False

In [4]:
# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 236.74it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:30<00:00,  2.59it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.159% |Test Loss: 2.292 | Test Acc: 12.248% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


#### Create New Client Class

In [5]:
import torch.nn.functional as F

from copy import deepcopy
from utils.torch_utils import *


class Adv_MixtureClient(MixtureClient):
    def __init__(
            self,
            learners_ensemble,
            train_iterator,
            val_iterator,
            test_iterator,
            logger,
            local_steps,
            tune_locally=False,
            adv_proportion=0,
            atk_params = None
    ):
        super(Adv_MixtureClient, self).__init__(
            learners_ensemble=learners_ensemble,
            train_iterator=train_iterator,
            val_iterator=val_iterator,
            test_iterator=test_iterator,
            logger=logger,
            local_steps=local_steps,
            tune_locally=tune_locally
        )

        self.adv_proportion = adv_proportion
        self.atk_params = atk_params
        
        # Make copy of dataset and set aside for adv training
        self.og_dataloader = copy.deepcopy(self.train_iterator) # Update self.train_loader every iteration
        
        # Add adversarial client 
        combined_model = self.combine_learners_ensemble()
        altered_dataloader = self.gen_customdataloader(self.og_dataloader)
        self.adv_nn = Adv_NN(combined_model, altered_dataloader)
        
    def gen_customdataloader(self, og_dataloader):
        # Combine Validation Data across all clients as test
        data_x = []
        data_y = []

        for (x,y,idx) in og_dataloader.dataset:
            data_x.append(x)
            data_y.append(y)

        data_x = torch.stack(data_x)
        data_y = torch.stack(data_y)
        dataloader = Custom_Dataloader(data_x, data_y)
        
        return dataloader
    
    def combine_learners_ensemble(self):

        # This is where the models are stored -- one for each mixture --> learner.model for nn
        hypotheses = self.learners_ensemble.learners

        # obtain the state dict for each of the weights 
        weights_h = []

        model_weights = self.learners_ensemble.learners_weights
        
        for h in hypotheses:
            weights_h += [h.model.state_dict()]
        
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            htemp = model_weights[0]*weights_h[0][key]
            for i in range(1,len(model_weights)):
                htemp+=model_weights[i]*weights_h[i][key]
            new_weight_dict[key] = htemp
        new_model.load_state_dict(new_weight_dict)
        
        return new_model
    
    def update_advnn(self):
        # reassign weights after trained
        self.adv_nn = self.combine_learnes_ensemble()
        return
    
    def generate_adversarial_data(self):
        # Generate adversarial datapoints while recognizing idx of sampled without replacement
        
        # Draw random idx without replacement 
        num_datapoints = self.train_iterator.dataset.targets.shape[0]
        sample_size = int(np.ceil(num_datapoints * self.adv_proportion))
        sample = np.random.choice(a=num_datapoints, size=sample_size)
        x_data = self.adv_nn.dataloader.x_data[sample]
        y_data = self.adv_nn.dataloader.y_data[sample]
        
        self.adv_nn.pgd_sub(self.atk_params, x_data.cuda(), y_data.cuda())
        x_adv = self.adv_nn.x_adv
        
        return sample, x_adv
    
    def assign_advdataset(self):
        # convert dataset to normed and replace specific datapoints
        
        # Flush current used dataset with original
        self.train_iterator = copy.deepcopy(self.og_dataloader)
        
        # adversarial datasets loop, adjust normed and push 
        sample_id, x_adv = self.generate_adversarial_data()
        
        for i in range(sample_id.shape[0]):
            idx = sample_id[i]
            x_val_normed = x_adv[i]
            x_val_unnorm = unnormalize_cifar10(x_val_normed)
        
            self.train_iterator.dataset.data[idx] = x_val_unnorm
        
        self.train_loader = iter(self.train_iterator)
        
        return
    
    

#### Pre-req of Initializing Client

In [6]:
data_dir = get_data_dir(args_.experiment)
root_path = os.path.join(data_dir, "train")

if "logs_root" in args_:
    logs_root = args_.logs_root
else:
    logs_root = os.path.join("logs", args_to_string(args_))

print("===> Building data iterators..")
train_iterators, val_iterators, test_iterators =\
    get_loaders(
        type_=LOADER_TYPE[args_.experiment],
        root_path=root_path,
        batch_size=args_.bz,
        is_validation=args_.validation
    )

print("===> Initializing clients..")
clients_ = []
for task_id, (train_iterator, val_iterator, test_iterator) in \
        enumerate(tqdm(zip(train_iterators, val_iterators, test_iterators), total=len(train_iterators))):

    if train_iterator is None or test_iterator is None:
        continue

    learners_ensemble =\
        get_learners_ensemble(
            n_learners=args_.n_learners,
            name=args_.experiment,
            device=args_.device,
            optimizer_name=args_.optimizer,
            scheduler_name=args_.lr_scheduler,
            initial_lr=args_.lr,
            input_dim=args_.input_dimension,
            output_dim=args_.output_dimension,
            n_rounds=args_.n_rounds,
            seed=args_.seed,
            mu=args_.mu
        )

    logs_path = os.path.join(logs_root, "task_{}".format(task_id))
    os.makedirs(logs_path, exist_ok=True)
    logger = SummaryWriter(logs_path)

===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 205.47it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:27<00:00,  2.94it/s]


In [7]:
atk_params = PGD_Params()

atk_params.set_params(batch_size=500, iteration = 30,
                   target = -1, x_val_min = -2, x_val_max = 2,
                   step_size = 0.05, step_norm = "inf", eps = 4.5, eps_norm = "inf")

client1 = Adv_MixtureClient(
            learners_ensemble=learners_ensemble,
            train_iterator=train_iterator,
            val_iterator=val_iterator,
            test_iterator=test_iterator,
            logger=logger,
            local_steps=args_.local_steps,
            tune_locally=args_.locally_tune_clients,
            adv_proportion = 0.3,
            atk_params=atk_params)

In [9]:
client1.assign_advdataset()